In [1]:
import os
import shutil
import pandas as pd
import re
import cv2
import numpy as np

In [2]:
def get_box(image_file):
    '''
    Used to get width and height of object given the track image filename
    image_file: name of image file
    '''
    # Extract numeric values using regular expression, last two represent width and height
    numeric_values = [int(match) for match in re.findall(r'\d+', image_file)]
    
    rect_w = numeric_values[-1]
    rect_h = numeric_values[-2]
    
    return rect_w, rect_h

In [3]:
def get_coords(image_file):
    '''
    Used to get x and y coords of object given the track image filename
    image_file: name of image file
    '''
    # Extract numeric values using regular expression, last two represent width and height
    numeric_values = [int(match) for match in re.findall(r'\d+', image_file)]
    
    x = numeric_values[9]
    y = numeric_values[10]
    
    return x, y

In [4]:
def get_category(row):
    '''
    Used to get the category label of a track image
    '''
    cats = ['bird','cable','panel','plant','car','human','other_animal','insect','aircraft','other','unknown']
    for i in range(len(cats)):
        if isinstance(row, pd.Series): # to handle type issue
            if row[cats[i]] == 1:
                category = i
                return category
        if isinstance(row, pd.DataFrame): # to handle type issue
            if row[cats[i]].item() == 1:
                category = i
                return category

In [5]:
def compare_obj_distance(row1,row2,crop_size):
    '''
    Given some rows of the raw dataframe, compare their coord distances
    '''
    hor_dist = abs(row1['x'] - row2['x'])
    vert_dist = abs(row1['y'] - row2['y'])

    # check if center coords of image in the same frame is within crop distance
    if hor_dist >= crop_size/2 or vert_dist >= crop_size/2:
        return False
    else:
        return row2['image_file']

In [6]:
def get_labels_same_frame(non_unique_rows):
    crop_size = 400 # set depending on your track image size NxN 
    diff_window = []
    same_window = {}

    for video_and_frame in non_unique_rows['key'].unique():
        tmp = non_unique_rows[non_unique_rows['key'] == video_and_frame]

        # iterate through each row and compare it to all other rows
        for i, row1 in tmp.iterrows():
            check_diff = []
            for j, row2 in tmp.iterrows():
                if i != j: # To avoid comparing a row with itself
                    check_diff.append(compare_obj_distance(row1, row2, crop_size))
            
            # if no center coords are within bounds of each other, append to diff_window
            # if center coords are within bounds of each other, will have to have multiple yolo labels in a file, append to dict
            if all(not value for value in check_diff):
                diff_window.append(row1['image_file'])
            else:
                # Use a list comprehension to extract strings
                obj_list = [item for item in check_diff if isinstance(item, str)]
                same_window[row1['image_file']] = obj_list

    return diff_window, same_window
                

In [7]:
def create_label(df, output_dir, dim, img_list = None):
    '''
    Used to generate yolo formatted labels 
    data_file: Path to structured datafile. Tracks should be labeled beforehand.
    output_dir: Output directory of labels.
    dim: Dimension of images. The way track images are generated, should be square with the obj being tracked in the center.
    img_list: optional arg, used for when list of imgs that exist on same frame but are not in same window has been provided.
    '''
    count = 0
    if img_list is not None:
        df = df[df['image_file'].isin(img_list)]
    for index, row in df.iterrows():
        label_name = row['image_file'].replace('.png','.txt')

        output_file = output_dir + '/' + label_name

        x = dim // 2
        y = dim // 2
        rect_w, rect_h = get_box(row['image_file'])
        category = get_category(row)

        # Write the list to the file, each element as a new line
        with open(output_file, "w") as file:
            print_buffer = []
            # normalize height for yolo format
            x /= dim
            y /= dim 
            rect_w /= dim
            rect_h /= dim
            print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(category, x, y, rect_w, rect_h)) # class, centerx, centery, width, height
            
            file.write("\n".join(print_buffer))
            count += 1
    print(f'{count} labels generated.')

In [8]:
def create_label_same_window(df,output_dir, dim, img_dct):
    count = 0
    for key in img_dct:
        # turn key value pair into a list
        img_list = img_dct[key]
        img_list.append(key)

        label_name = key.replace('.png','.txt')
        output_file = output_dir + '/' + label_name

        with open(output_file, "w") as file:
            print_buffer = []
            cen_x, cen_y = get_coords(key) # save original center coords, will use to recalculate true coords
            for i in img_list:
                if i == key:
                    x = dim // 2
                    y = dim // 2
                else:
                    alt_x, alt_y = get_coords(i) # current img center coords. 
                    # calculate offset
                    dif_x = alt_x - cen_x
                    dif_y = alt_y - cen_y
                    # using center of the base image (since obj tracked is always in center), calculate coords of each additional object detected in window
                    x = (dim // 2) + dif_x
                    y = (dim // 2) + dif_y

                row = df[df['image_file'] == i]
                rect_w, rect_h = get_box(i)
                category = get_category(row)

                # normalize for yolo format
                x /= dim
                y /= dim
                rect_w /= dim
                rect_h /= dim
                print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(category, x, y, rect_w, rect_h)) # class, centerx, centery, width, height
            file.write("\n".join(print_buffer))
            count += 1
    print(f'{count} labels generated.')

In [9]:
def gen_labels(data_file, output_dir, dim=400):
    '''
    Used to generate yolo formatted labels 
    data_file: Path to structured datafile. Tracks should be labeled beforehand.
    output_dir: Output directory of labels.
    dim: Dimension of images. The way track images are generated, should be square with the obj being tracked in the center.
    '''
    # Ensure the destination directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    df = pd.read_csv(data_file)
    
    # Find non-unique values based on two frame and video name
    # create key column so that we can iterate through rows that have the same video and frame number later on.
    df['key'] = df['video_dir'] + df['frame'].astype(str)
    # NOTE 2024-01-23: Frame numbering has issue at the moment, but can still be used to tell if multiple object in a frame
    # TODO: FIND better way to create labels for potential overlap
    non_unique_rows = df[df.duplicated(['video_dir', 'frame'], keep=False)]

    if len(non_unique_rows) != 0:
        diff_window, same_window = get_labels_same_frame(non_unique_rows)

    # Drop non-unique rows from the original DataFrame
    df = df.drop_duplicates(['video_dir', 'frame'], keep=False)

    create_label(df, output_dir, dim) # path 1: obj without another in the same frame
    create_label(non_unique_rows, output_dir, dim, diff_window) # path 2: obj w another in the same frame but not the same window
    create_label_same_window(non_unique_rows, output_dir, dim, same_window) # path 3: obj w another in the same frame and same window

    file_list = list(df['image_file']) + list(non_unique_rows['image_file'])

    # return list of image files so that we can copy the correct ones over
    return file_list

In [10]:
def check_image_valid(img_path, threshold):
    '''
    Check if an image has too many black pixels. Using our tracker algo sometimes results in images that track
    and object outside of the video frame.
    This function was built to handle such edge cases.
    It's a bit conservative, sometimes drops valid images that are close to bounds.
    img_path: directory path to image file
    thresholde: boundary of percentage of black pixels before an image is excluded.
    '''
    # Read the image
    img = cv2.imread(img_path)

    # Check if the image is successfully loaded
    if img is not None:
        # Convert the image to grayscale
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Count the number of black pixels
        black_pixel_count = np.sum(gray_img == 0)
        
        # Calculate the percentage of black pixels
        total_pixels = img.size
        percentage_black_pixels = (black_pixel_count / total_pixels) * 100

        # Check if the percentage exceeds the threshold percentage specified
        if percentage_black_pixels > threshold:
            return False
    else:
        print(f"Error: Unable to open the image at {img_path}")
        return False

    return True

In [11]:
def copy_png(target_dir, output_dir, imgs):
    '''
    Copies png files to a target directory. Only copies files if in specified list.
    target_dir: root directory of tracks/track images (doesn't matter too much since only copies png)
    output_dir: output directory of images. Single dir for all images.
    imgs: list of images to copy over
    '''
    # Ensure the destination directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    thresh = 15

    count = 0
    count_drop = 0

    # Recursively iterate through all files in the source directory and its subdirectories
    for root, dirs, files in os.walk(target_dir):
        for file in files:
            # check if file is in list of images to copy over
            if file.endswith(".png") and file in imgs:
                source_path = os.path.join(root, file)
                destination_path = os.path.join(output_dir, file)

                # check if image being copied is a valid one
                # This function/condition deals with edge case where obj detected is outside video frame
                if check_image_valid(source_path,thresh) == True:
                    shutil.copy2(source_path, destination_path)
                    count += 1
                else:
                    count_drop += 1
                    pass
    print(f'{count_drop} images dropped due to containing black pixels greater than {thresh}% threshold.')
    print(f'{count} images copied over.')

In [12]:
def compare_and_remove(labels_dir, imgs_dir):
    '''
    Some images will be dropped when copied over.
    This function performs some reconciliation to match labels to image files.
    '''

    count = 0

    # Get the list of files in each folder without extensions
    files1 = {os.path.splitext(file)[0] for file in os.listdir(labels_dir)}
    files2 = {os.path.splitext(file)[0] for file in os.listdir(imgs_dir)}

    # Identify files present in labels_dir but not in imgs_dir
    files_to_remove = files1 - files2

    # Remove files from labels_dir that are not present in imgs_dir
    for file_name in files_to_remove:
        file_path = os.path.join(labels_dir, file_name+".txt")
        os.remove(file_path)
        count += 1
    
    print(f'Removed {count} labels due to image exclusion.')

In [13]:
def count_files(directory_path):
    try:
        # List all files in the directory
        files = os.listdir(directory_path)
        
        # Filter out directories and count the remaining files
        file_count = sum(os.path.isfile(os.path.join(directory_path, file)) for file in files)

        print(f'{file_count} files in {directory_path}')
        
    except OSError as e:
        print(f"Error reading directory {directory_path}: {e}")
        return None

In [14]:
data_file = "C:/Users/Aaron/Desktop/uchicago-aviansolar-detect-track/data/400x400/merged_small_20240203.csv"
output_dir_labels = "C:/Users/Aaron/Desktop/uchicago-aviansolar-detect-track/custom/labels"

target_dir = "C:/Users/Aaron/Desktop/uchicago-aviansolar-detect-track/data/400x400/" # root dir containing track images
output_dir_imgs = "C:/Users/Aaron/Desktop/uchicago-aviansolar-detect-track/custom/images"

labels = gen_labels(data_file, output_dir_labels)
copy_png(target_dir, output_dir_imgs, labels)
compare_and_remove(output_dir_labels, output_dir_imgs)
count_files(output_dir_imgs)
count_files(output_dir_labels)


3648 labels generated.
1237 labels generated.
324 labels generated.
132 images dropped due to containing black pixels greater than 15% threshold.
5077 images copied over.
Removed 132 labels due to image exclusion.
5077 files in C:/Users/Aaron/Desktop/uchicago-aviansolar-detect-track/custom/images
5077 files in C:/Users/Aaron/Desktop/uchicago-aviansolar-detect-track/custom/labels
